In [32]:
import pandas as pd
from openpyxl import load_workbook
pd.reset_option('display.float_format')
df_inventory = pd.read_excel(r"K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0307\inventory.xlsx")
df_movement = pd.read_excel(r"K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0307\movement.xlsx")
df_ffca= pd.read_excel(r"K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0307\FFCA.xlsx")
df_qv= pd.read_excel(r"K:\FINANCE\Concs\_ Yuyan\test_reporter\BB_report_test_files\0307\QV.xlsx")

In [64]:
cols_ffca = ["Purchase Assignment ID" , 'Current Value',  "Inventory ID", "Counterparty of Sales", "Financing Bank facility", "Inventory Status", "QTY"]
cols_qv = ["Assignment ID" , "Country" ]
df_ffca_f1 = pd.DataFrame(df_ffca, columns=cols_ffca)
df_qv_f1 = pd.DataFrame(df_qv, columns=cols_qv)

cols_inventory_transit = ['INVENTORY ID', 'MOVEMENT ID', 'PURCHASE ASSIGNMENT', 'PRODUCT', 'QUALITY', \
    'FINANCING BANK-TF', 'FINANCING FACILITY TYPE-TF', 'OPERATIONS OWNER', 'DRY WEIGHT', 'WET WEIGHT']

cols_inventory_storage = ['INVENTORY ID', 'PURCHASE ASSIGNMENT', 'PRODUCT', 'QUALITY', 'TRANSPORT STATUS', 'LOCATION', 'PURCHASE COUNTERPARTY', 'DRY WEIGHT', 'MOISTURE', 'WET WEIGHT', \
    'FINANCING BANK-TF', 'FINANCING FACILITY TYPE-TF', 'OPERATIONS OWNER']

cols_movement_transit = ['ID', 'MOVEMENT PURPOSE', 'LOAD', 'LOAD DATE', 'DESTINATION', 'DESTINATION DATE',  \
    'WET WEIGHT', 'DRY WEIGHT', 'UOM']

df_inventory_f1 = pd.DataFrame(df_inventory, columns=cols_inventory_transit)
df_inventory_f2 = pd.DataFrame(df_inventory, columns=cols_inventory_storage)
df_movement_f1 = pd.DataFrame(df_movement, columns=cols_movement_transit)

df_movement_f1.loc[:, 'Load Country'] = df_movement_f1['LOAD'].apply(lambda st: st[st.rfind("(")+1:st.rfind(")")])
df_movement_f1.loc[:, 'Delivery Country'] = df_movement_f1['DESTINATION'].apply(lambda st: st[st.rfind("(")+1:st.find(")")])

ffca_qv = pd.merge(df_ffca_f1, df_qv_f1, left_on='Purchase Assignment ID', right_on='Assignment ID', how='outer')




In [65]:
#delivery
def location(a):
    return a == "China" or a == 'Taiwan' \
        or a == 'Malaysia' or a == 'Spain' \


ffca_qv_storage = ffca_qv.loc[ffca_qv['Country'].apply(location)]

summary_storage = pd.merge(df_inventory_f2, ffca_qv_storage, left_on=['INVENTORY ID'], right_on=['Inventory ID'])
summary_storage.to_excel(r"C:\Users\Yuyan.Lu\Desktop\APAC inventory\test.xlsx", sheet_name='summary storage')


In [62]:
# def destination(a):
#     return a == "Johor-Henry Bath Singapore Pte. Ltd-Warehouse 9 (Malaysia)" or a == "Johor - Henry Bath Singapore warehouse 6B1 - Stirling International Logistic Sdn Bhd (Malaysia)"
summary_transit = pd.merge(df_inventory_f1, df_movement_f1, left_on=['MOVEMENT ID'], right_on=['ID'])
summary_transit = pd.merge(summary_transit, ffca_qv, left_on='INVENTORY ID', right_on='Inventory ID')
# summary_transit = summary_transit.loc[summary_transit['DESTINATION'].apply(destination)]


In [63]:

writer = pd.ExcelWriter(r"C:\Users\Yuyan.Lu\Desktop\APAC inventory\test.xlsx", engine='openpyxl')
book = load_workbook(writer.path)
writer.book = book
summary_transit.to_excel(excel_writer=writer, sheet_name='summary intransit')
writer.save()
writer.close()